In [ ]:
import geopandas as gpd
import odc.geo.xr  # noqa: F401
from odc.stac import load
from pystac_client import Client
from shapely import box

import sarsen

In [ ]:
bbox = [146.2357, -43.6796, 147.147, -42.9305]
geometry = gpd.GeoSeries([box(*bbox)], crs="EPSG:4326")

geometry.explore()

In [ ]:
# Get some elevation

client = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
items = client.search(collections=["cop-dem-glo-30"], bbox=bbox).items()

data = load(
    items,
    bbox=bbox,
    resolution=10,
    output_crs="EPSG:28355",
    chunks=dict(x=2048, y=2048),
)

data.data.odc.write_cog("elevation.tif", overwrite=True)

In [ ]:
_ = sarsen.terrain_correction(
    sarsen.Sentinel1SarProduct(
        "data/S1A_IW_GRDH_1SDV_20230326T191836_20230326T191902_047819_05BECD_540C.SAFE/manifest.safe",
        measurement_group="IW/VV",
    ),
    interp_method="linear",
    dem_urlpath="data/tas_dem_10m.tif",
    output_urlpath="data/tas_vv_10.tif",
    correct_radiometry="gamma_bilinear",
    enable_dask_distributed=True,
)

_ = sarsen.terrain_correction(
    sarsen.Sentinel1SarProduct(
        "data/S1A_IW_GRDH_1SDV_20230326T191836_20230326T191902_047819_05BECD_540C.SAFE/manifest.safe",
        measurement_group="IW/VH",
    ),
    interp_method="linear",
    dem_urlpath="data/tas_dem_10m.tif",
    output_urlpath="data/tas_vh_10.tif",
    correct_radiometry="gamma_bilinear",
    enable_dask_distributed=True,
)

In [ ]:
# Using the COP DEM

_ = sarsen.terrain_correction(
    sarsen.Sentinel1SarProduct(
        "S1A_IW_GRDH_1SDV_20230326T191836_20230326T191902_047819_05BECD_540C.SAFE/manifest.safe",
        measurement_group="IW/VV",
    ),
    interp_method="linear",
    dem_urlpath="elevation.tif",
    output_urlpath="s1_g0_cop_vv.tif",
    correct_radiometry="gamma_bilinear",
)

_ = sarsen.terrain_correction(
    sarsen.Sentinel1SarProduct(
        "S1A_IW_GRDH_1SDV_20230326T191836_20230326T191902_047819_05BECD_540C.SAFE/manifest.safe",
        measurement_group="IW/VH",
    ),
    interp_method="linear",
    dem_urlpath="elevation.tif",
    output_urlpath="s1_g0_cop_vh.tif",
    correct_radiometry="gamma_bilinear",
)

In [ ]:
# Using the Tas DEM

_ = sarsen.terrain_correction(
    sarsen.Sentinel1SarProduct(
        "S1A_IW_GRDH_1SDV_20230326T191836_20230326T191902_047819_05BECD_540C.SAFE/manifest.safe",
        measurement_group="IW/VV",
    ),
    interp_method="linear",
    dem_urlpath="tas_dem_10m.tif",
    output_urlpath="s1_g0_vv.tif",
    correct_radiometry="gamma_nearest",
)

_ = sarsen.terrain_correction(
    sarsen.Sentinel1SarProduct(
        "S1A_IW_GRDH_1SDV_20230326T191836_20230326T191902_047819_05BECD_540C.SAFE/manifest.safe",
        measurement_group="IW/VH",
    ),
    interp_method="linear",
    dem_urlpath="tas_dem_10m.tif",
    output_urlpath="s1_g0_vh.tif",
    correct_radiometry="gamma_nearest",
)

In [ ]:
from planetary_computer import sign_url

s1_items = client.search(
    collections=["sentinel-1-rtc"], bbox=bbox, datetime="2023-03"
).items()

s1_data = load(
    s1_items,
    bbox=bbox,
    resolution=10,
    output_crs="EPSG:28355",
    chunks=dict(x=2048, y=2048),
    patch_url=sign_url,
)

s1_data.isel(time=-1).vv.odc.write_cog("s1_mspc_vv.tif")

In [ ]:
s1_data.isel(time=-1).vh.odc.write_cog("s1_mspc_vh.tif")